In [13]:
using DataFrames
using CSV
using ScatteredInterpolation
using CairoMakie
using AlgebraOfGraphics


function find_missing_columns(df::DataFrame)
    return [col for col in names(df) if any(ismissing.(df[!, col]))]
end

function remove_missing_columns!(df::DataFrame)
    missing_cols = find_missing_columns(df)
    select!(df, Not(missing_cols))
    return df
end

function load_isochrone_data(filepath::String)
    df = DataFrame(CSV.File(filepath, delim=' ', ignorerepeated=true, comment="#", silencewarnings=true))
    remove_missing_columns!(df)
    return df
end

function add_evolution_phase!(df::DataFrame)
    df.evol = zeros(nrow(df))
    for label in unique(df.label)
        idx = df.label .== label
        n_points = sum(idx)
        df[idx, :evol] = label .+ range(0, 1 - 1/n_points, length=n_points)
    end
    return df
end

function find_interval_uniform(A::Vector{T}, x::T) where T<:Real
    if x < first(A) || x > last(A)
        error("x está fuera del rango de A. x debe estar en [$(first(A)), $(last(A))].")
    end

    if x == last(A)
        return length(A) - 1
    end

    step = (last(A) - first(A)) / (length(A) - 1)
    i = floor(Int, (x - first(A)) / step) + 1
    return i
end

function find_nearest_isochrones(df::DataFrame, log_age::Real, metal::Real)
    age_vals = sort(unique(df.logAge))
    metal_vals = sort(unique(df.MH))

    age_idx = find_interval_uniform(age_vals, log_age)
    metal_idx = find_interval_uniform(metal_vals, metal)

    age_nodes = age_vals[age_idx:age_idx+1]
    metal_nodes = metal_vals[metal_idx:metal_idx+1]

    return df[(df.logAge .∈ [age_nodes]) .& (df.MH .∈ [metal_nodes]), :]
end

function interpolate_isochrone(df::DataFrame, target_age::Real, target_metallicity::Real)
    log_age = log10(target_age)
    nearest = find_nearest_isochrones(df, log_age, target_metallicity)
    add_evolution_phase!(nearest)

    points = Matrix(nearest[:, [:logAge, :MH, :evol]])
    mag_cols = setdiff(names(nearest), [:logAge, :MH, :label, :evol])

    max_label = maximum(nearest.label)
    @show max_label
    evol_points = range(0, max_label + 0.99, length=1_000*(max_label+1))

    targets = hcat(
        fill(log_age, length(evol_points)),
        fill(target_metallicity, length(evol_points)),
        collect(evol_points)
    )
    @show floor.(Int, evol_points)
    result = DataFrame(
        evol = evol_points,
        logAge = fill(log_age, length(evol_points)),
        MH = fill(target_metallicity, length(evol_points)),
        label = floor.(Int, evol_points)
    )

    for col in mag_cols
        itp = interpolate(Multiquadratic(), points', nearest[:, col])
        result[!, col] = evaluate(itp, targets')
    end

    return dropmissing(result)
end

function plot_isochrone_cmd(df::DataFrame, telescope::Symbol, file::String)
    # Filtrar datos donde label <= 5
    df_filtered = filter(row -> row.label <= 5, df)

    size_inches = (3 * 3, 3 * 3)
    size_pt = 72 .* size_inches
    fig = Figure(size = size_pt, fontsize = 30)

    # Paleta discreta
    discrete_colors = :tab10

    if telescope == :Gaia
        df_filtered.color = df_filtered."BP-RP"
        plt = data(df_filtered) *
              mapping(:color => L"BP-RP", :Gaia_G_EDR3 => L"G") *
              visual(Lines)
        draw!(fig, plt, axis = (; yreversed = true))

    elseif telescope == :Subaru
        df_filtered.color = df_filtered.gmag - df_filtered.rmag

        # Mapeo directo de labels a strings (sin conversión categórica)
        phase_names = Dict(1=>"MS", 2=>"SGB", 3=>"RGB", 4=>"HeB", 5=>"EAGB")
        df_filtered.phase = [phase_names[l] for l in df_filtered.label]

        plt = data(df_filtered) *
              mapping(:color => L"g-r", :gmag => L"g",
                     color=:phase => "Evolutionary Phase",
                     linestyle=:phase) *
              visual(Lines, linewidth=2)

        draw!(fig, plt,
              axis = (; yreversed = true),
              legend = (position = :right, titleposition = :left),
              theme = Theme(palette = (color = discrete_colors,)))
    else
        error("Telescope $telescope not available for CMD plot.")
    end

    save(file, fig, pt_per_unit = 1)
    return fig
end

"""Example use with Subaru data."""
function example_interpolate_isochrone()
    filter = "hsc"
    file_artif = "../artifacts/isochrones/parsec/$filter/family_MH_-2.2_0.5_logAge_9.2_10.3.dat"
    df_artif = load_isochrone_data(file_artif)
    file_plot = "../plots/isochrone_$filter.pdf"

    target_age = 10.0^9.8
    target_metallicity = 0.0

    isochrone_df = interpolate_isochrone(df_artif, target_age, target_metallicity)
    # fig = plot_isochrone_cmd(isochrone_df, :Subaru, file_plot)
    # println("Isócrona interpolada y graficada para Subaru.")

    return isochrone_df
end


example_interpolate_isochrone

In [14]:
df = example_interpolate_isochrone()

max_label = 9
floor.(Int, evol_points) = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Row,evol,logAge,MH,label,Zini,Mini,int_IMF,Mass,logL,logTe,logg,McoreTP,C_O,period0,period1,period2,period3,period4,pmode,Mloss,tau1m,X,Y,Xc,Xn,Xo,Cexcess,Z,\t,mbolmag,gmag,rmag,imag,zmag,ymag,NB816mag
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.273878,10.0773,-0.0229421,80.6469,0.01445,129476.0,145928.0,99177.8,1.13722e6,-46466.0,-1.21645e6,1.60381e5,-10457.6,1.17333e8,-2.10463e8,-1.31454e8,-1.06536e8,-8.72102e7,-6.64214e6,-1.31034,54620.9,-2804.86,2481.79,-54.1772,-1.73507,5.59943,-1.02746,49.7941,-2.83418e6,6.61091e5,7.43351e5,-1.12932e6,-1.81784e6,-2.28348e6,-1.80072e6,-2.12822e6
2,0.274,10.0765,-0.0229217,80.5297,0.0144493,1.29228e5,1.45638e5,98994.0,1.13479e6,-46366.3,-1.21382e6,1.60056e5,-10438.2,1.17089e8,-2.10036e8,-1.31188e8,-1.06321e8,-8.70334e7,-6.62877e6,-1.30781,54521.5,-2801.48,2478.8,-54.0452,-1.72155,5.66041,-1.02737,49.8542,-2.82811e6,6.59935e5,7.42115e5,-1.12672e6,-1.81384e6,-2.27852e6,-1.79676e6,-2.12358e6
3,0.274123,10.0756,-0.0229013,80.4115,0.0144487,1.28979e5,1.45348e5,98810.0,1.13235e6,-46266.5,-1.21119e6,159731.0,-10418.8,1.16844e8,-2.09609e8,-1.30921e8,-1.06105e8,-8.68567e7,-6.6154e6,-1.30528,54422.1,-2798.06,2475.79,-53.9136,-1.70792,5.7205,-1.02729,49.9137,-2.82203e6,6.58763e5,7.40861e5,-1.12414e6,-1.80984e6,-2.27357e6,-1.7928e6,-2.11893e6
4,0.274247,10.0747,-0.0228809,80.2943,0.014448,1.28731e5,1.45058e5,98626.4,1.12992e6,-46166.9,-1.20856e6,1.59406e5,-10399.4,1.16601e8,-2.09182e8,-1.30655e8,-1.05888e8,-8.66797e7,-6.60202e6,-1.30274,54322.6,-2794.66,2472.8,-53.7816,-1.69411,5.78183,-1.0272,49.9753,-2.81596e6,6.57611e5,7.3963e5,-1.12154e6,-1.80584e6,-2.26861e6,-1.78883e6,-2.11429e6
5,0.274372,10.0739,-0.0228605,80.1764,0.0144473,1.28482e5,1.44768e5,98442.3,1.12748e6,-46067.0,-1.20592e6,1.5908e5,-10380.0,1.16356e8,-2.08755e8,-1.30388e8,-1.05672e8,-8.65025e7,-6.58863e6,-1.30021,54223.0,-2791.25,2469.8,-53.6493,-1.68044,5.84274,-1.02712,50.0354,-2.80988e6,6.56448e5,7.38387e5,-1.11894e6,-1.80183e6,-2.26364e6,-1.78487e6,-2.10964e6
6,0.274499,10.073,-0.0228401,80.0589,0.0144467,1.28234e5,1.44477e5,98258.4,1.12504e6,-45967.1,-1.20329e6,1.58755e5,-10360.6,1.16112e8,-2.08327e8,-1.30121e8,-1.05456e8,-8.63254e7,-6.57523e6,-1.29767,54123.6,-2787.87,2466.82,-53.517,-1.66662,5.90422,-1.02703,50.0982,-2.8038e6,6.55293e5,737149.0,-1.11635e6,-1.79782e6,-2.25868e6,-1.7809e6,-2.10499e6
7,0.274628,10.0721,-0.0228197,79.941,0.014446,1.27985e5,144187.0,98074.2,1.1226e6,-45867.2,-1.20065e6,1.58429e5,-10341.1,1.15867e8,-2.07899e8,-1.29854e8,-1.05239e8,-8.61481e7,-6.56183e6,-1.29513,54023.9,-2784.47,2463.83,-53.3847,-1.65285,5.96557,-1.02695,50.1599,-2.79771e6,6.54129e5,7.35905e5,-1.11375e6,-1.79381e6,-2.25371e6,-1.77693e6,-2.10033e6
8,0.274758,10.0713,-0.0227992,79.8227,0.0144454,1.27735e5,143896.0,97889.7,1.12015e6,-45767.0,-1.19801e6,1.58102e5,-10321.7,1.15622e8,-2.0747e8,-1.29586e8,-1.05022e8,-8.59706e7,-6.54839e6,-1.29258,53924.1,-2781.09,2460.85,-53.2522,-1.63881,6.02715,-1.02686,50.2221,-2.79161e6,6.52961e5,7.34657e5,-1.11114e6,-1.78979e6,-2.24873e6,-1.77295e6,-2.09567e6
9,0.274889,10.0704,-0.0227788,79.7049,0.0144447,127486.0,143605.0,97705.3,1.11771e6,-45667.0,-1.19537e6,1.57776e5,-10302.2,1.15377e8,-2.07041e8,-1.29318e8,-1.04805e8,-8.57929e7,-6.53496e6,-1.29004,53824.2,-2777.69,2457.86,-53.1196,-1.62494,6.08873,-1.02678,50.2842,-2.78552e6,6.51804e5,7.3342e5,-1.10854e6,-1.78577e6,-2.24375e6,-1.76897e6,-2.09101e6


In [15]:
df.label

10000-element Vector{Float64}:
 80.6468722564764
 80.52969473394609
 80.41148339557867
 80.29426003091498
 80.17644726262569
 80.05886906531009
 79.94104478710273
 79.82272838001984
 79.70489259185439
 79.58648265880214
  ⋮
  5.2819038111223655
  5.274428159387256
  5.267291006277432
  5.258958385181355
  5.251040116932074
  5.242871032337767
  5.235692834063208
  5.228263380070032
  5.21998092068635